In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
# import tensorflow_io as tfio

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers

In [ ]:
BATCH_SIZE=32
IMG_SIZE=(1024, 1024)
SEED=123

In [ ]:
train_dataset = image_dataset_from_directory("train", shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE, validation_split=0.2, subset="training", seed=SEED)

In [ ]:
validation_dataset = image_dataset_from_directory("train", shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE, validation_split=0.2, subset="validation", seed=SEED)

In [ ]:
test_dataset = image_dataset_from_directory("test", shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)

In [ ]:
class_names = train_dataset.class_names

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

num_classes = len(class_names)
IMG_SHAPE = IMG_SIZE + (3,)

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip('horizontal', input_shape=IMG_SHAPE),
  layers.experimental.preprocessing.RandomFlip('vertical', input_shape=IMG_SHAPE),
  layers.experimental.preprocessing.RandomRotation(0.2),
  layers.experimental.preprocessing.RandomZoom(0.1),
])

In [ ]:
#Custom Model
activation = 'relu'
base_model = tf.keras.Sequential([
  layers.Conv2D(16, 3, padding='same'),
  layers.BatchNormalization(),
  layers.Activation(activation),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same'),
  layers.BatchNormalization(),
  layers.Activation(activation),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same'),
  layers.BatchNormalization(),
  layers.Activation(activation),
  layers.MaxPooling2D(),
  layers.Conv2D(128, 3, padding='same'),
  layers.BatchNormalization(),
  layers.Activation(activation),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation=activation),
  layers.Dropout(0.2),
  layers.Dense(num_classes)
])

In [ ]:
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

In [ ]:
base_model.summary()

In [ ]:
class Colorspace(layers.Layer):
    def __init__(self, colorspace, name=None, **kwargs):
        self.colorspace = colorspace
        super(Colorspace, self).__init__(name=name, **kwargs)
        
    def call(self, inputs):
        return self.colorspace(inputs)
        

In [ ]:
inputs = tf.keras.Input(shape=IMG_SHAPE)
x = data_augmentation(inputs)

process = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.Rescaling(1./255), # Rescale [0,1] for rgb processing
    Colorspace(colorspace = tf.image.rgb_to_hsv),
])

x = process(x)
x = base_model(x)
model = tf.keras.Model(inputs, x)

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset,
                    epochs=200,
                    validation_data=validation_dataset)